In [ ]:
# need to add dataset to a folder in google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Add the RITE dataset to your google drive for easy access

# Installs and imports

In [ ]:
! pip install SimpleITK
! pip install tensorflow-addons

     |████████████████████████████████| 48.4 MB 2.2 MB/s 
     |████████████████████████████████| 1.1 MB 11.0 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os
import datetime
from PIL import Image
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import SimpleITK as sitk
import pickle
from tensorflow import saved_model as sm

# Important functions

In [ ]:
def import_data_numpy(path):
    """
    Imports png images and returns as a list of tensors to be used in training
    :param path: path to directory where images are
    :return: list of Tensors
    """
    tensor_list = []
    for file in sorted(os.listdir(path)):
        if file.endswith(".png") or file.endswith(".tif"):
            # Converts to RGB because the vessel images are black and white
            png_img = Image.open(os.path.join(path, file))
            np_arr = normalize(np.array(png_img.getdata()).reshape((128, 128,3)))
            # tensor = tf.convert_to_tensor(np_arr)
            # Encodes images as tensors and adds to the list
            tensor_list.append(np_arr)
        # Returns list of tensors to be used in model
    # tensor_list = np.asarray(tensor_list)
    return np.asarray(tensor_list)


In [ ]:
def import_data_numpy_mask(path):
    """
    Imports png images and returns as a list of tensors to be used in training
    :param path: path to directory where images are
    :return: list of Tensors
    """
    tensor_list = []
    for file in sorted(os.listdir(path)):
        if file.endswith(".png") or file.endswith(".tif"):
            # Converts to RGB because the vessel images are black and white
            png_img = Image.open(os.path.join(path, file)).convert('L')
            np_arr = normalize(np.array(png_img.getdata()).reshape((128, 128,1)))
            # tensor = tf.convert_to_tensor(np_arr)
            # Encodes images as tensors and adds to the list
            tensor_list.append(np_arr)
        # Returns list of tensors to be used in model
    # tensor_list = np.asarray(tensor_list)
    return np.asarray(tensor_list)

In [ ]:
def normalize(input_image):
  max = np.max(input_image)
  input_image = tf.cast(input_image, tf.float32) / max
  return input_image

In [ ]:
from keras import backend as K    #import backend for custom metric

def recall_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_metric(y_true, y_pred):
    precision = precision_metric(y_true, y_pred)
    recall = recall_metric(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def _encoder_block_unet(input, num_filters):
    """
    Encoder logic for U-Net
    :param input: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :return: KerasTensor
    """
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.MaxPool2D((2, 2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x


def _encoder_block_kinet(input, num_filters):
    """
    Encoder logic for Ki-Net
    :param input: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :return: KerasTensor
    """
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.UpSampling2D(size=(2, 2), interpolation="bilinear")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    p = layers.MaxPool2D((2, 2))(x)
    return x


def _decoder_block_unet(input, num_filters):
    """
    Decoder logic for U-Net
    :param input: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :return: KerasTensor
    """
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.UpSampling2D(size=(2, 2), interpolation="bilinear")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x


def _decoder_block_kinet(input, num_filters):
    """
    Decoder logic for Ki-net
    :param input: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :return: KerasTensor
    """
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.MaxPool2D((2, 2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    print(type(x))
    return x


def _crfb(x1, x2, num_filters, scale_factor):
    """
    CRFB is the cross residual fusion block which fuses the outputs at the specified layers
    from one model to the other, and returns the results as input for the next step.
    :param x1: KerasTensor
    :param x2: KerasTensor
    :param num_filters: Int - number of output filters in convolution
    :param scale_factor: Float - scale factor for resizing (upsampling/downsampling) of output
    :return: KerasTensor
    """
    new_w = int(x2.shape[1] * scale_factor)
    new_h = int(x2.shape[2] * scale_factor)
    out = layers.Conv2D(num_filters, 3, padding="same")(x2)
    out = layers.Activation("relu")(out)
    out = tf.image.resize(out, [new_h, new_w])
    output = layers.Add()([x1, out])
    return output

In [ ]:
def kiunet(input_shape, num_classes):
    """
    Architecture of the KiU-Net model, which combines the Ki-Net and U-Net architecture
    :param input_shape: Tuple of the shape of the input
    :return: keras.Model
    """
    inputs = layers.Input(shape=input_shape)

    # ENCODER BLOCK #

    s1 = _encoder_block_unet(inputs, 16)  # U NET ENCODER
    k1 = _encoder_block_kinet(inputs, 16)  # KINET ENCODER
    
    u1 = _crfb(s1, k1, 16, 0.25)  # CRFB U1 UNET
    o1 = _crfb(k1, s1, 16, 4)  # CRFB O1 KINET
    
    s2 = _encoder_block_unet(u1, 32)  # UNET ENCODER
    k2 = _encoder_block_kinet(o1, 32)  # KINET ENCODER
    
    u2 = _crfb(s2, k2, 32, 0.0625)  # CRFB U2 UNET
    o2 = _crfb(k2, s2, 32, 16)  # CRFB O2 KINET
    

    s3 = _encoder_block_unet(u2, 64)  # UNET ENCODER
    k3 = _encoder_block_kinet(o2, 64)  # KINET ENCODER
    

    u3 = _crfb(s3, k3, 64, 0.015625)  # CRFB U3 UNET
    o3 = _crfb(k3, s2, 64, 32)  # CRFB O3 KINET
    

    # DECODER BLOCK #

    d1_u = _decoder_block_unet(u3, 32)  # UNET DECODER
    d1_k = _decoder_block_kinet(o3, 32)  # KINET DECODER
    

    d_u1 = _crfb(d1_u, d1_k, 32, 0.0625)  # CRFB D_U1 UNET
    
    d_o1 = _crfb(d1_k, d1_u, 32, 16)  # CRFB D_O1 KINET
    

    out = layers.Add()([d_u1, s2])  # CONCATENTATION D_U1 & S2 UNET
    out1 = layers.Add()([d_o1, k2])  # CONCATENTATION D_O1 & K2 KINET
    

    d2_u = _decoder_block_unet(out, 16)  # UNET DECODER
    d2_k = _decoder_block_kinet(out1, 16)  # KINET DECODER
   

    d_u2 = _crfb(d2_u, d2_k, 16, 0.25)  # CRFB D_U2 UNET
    d_o2 = _crfb(d2_k, d2_u, 16, 4)  # CRFB D_O2 KINET
    

    out = layers.Add()([d_u2, s1])  # CONCATENATION D_U2 & S1 UNET
    out1 = layers.Add()([d_o2, k1])  # CONCATENATION D_O2 & K2 KINET
    

    d3_u = _decoder_block_unet(out, 8)  # UNET DECODER
    d3_k = _decoder_block_kinet(out1, 8)  # KINET DECODER
    

    out = layers.Add()([d3_u, d3_k])  # FINAL CONCATENATION OUTPUT FROM UNET AND KINET

    out = layers.Conv2D(num_classes, 1, padding="valid", activation="relu")(out)  # FINAL CONVOLUTIONAL LAYER
    kiunet_model = keras.Model(inputs, out, name="KiU-Net")  # MODEL

    return kiunet_model

In [ ]:
def unet(input_shape, num_classes):
    """
    Architecture of the U-Net model
    :param input_shape: Tuple of the shape of the input
    :return: keras.Model
    """
    inputs = layers.Input(shape=input_shape)
    print(input_shape)
    # ENCODER BLOCK #

    s1 = _encoder_block_unet(inputs, 32)  # U NET ENCODER
    s2 = _encoder_block_unet(s1, 64)  # UNET ENCODER
    s3 = _encoder_block_unet(s2, 128)  # UNET ENCODER
    s4 = _encoder_block_unet(s3, 256)  # UNET ENCODER
    out = _encoder_block_unet(s4, 512)  # UNET ENCODER

    # DECODER BLOCK #

    out = _decoder_block_unet(out, 256)
    out = layers.Add()([out, s4])
    out = _decoder_block_unet(out, 128)
    out = layers.Add()([out, s3])
    out = _decoder_block_unet(out, 64)  # UNET DECODER
    out = layers.Add()([out, s2])  # CONCATENTATION D_U1 & S2 UNET
    out = _decoder_block_unet(out, 32)  # UNET DECODER
    out = layers.Add()([out, s1])  # CONCATENATION D_U2 & S1 UNET
    out = _decoder_block_unet(out, num_classes)  # UNET DECODER
    unet_model = tf.keras.Model(inputs, out, name="U-Net")  # MODEL

    return unet_model

In [ ]:
def segnet(input_shape, num_classes):
  inputs = layers.Input(shape=input_shape)
  s0 = _encoder_block_unet(inputs, 16)
  s1 = _encoder_block_unet(s0, 32)
  s2 = _encoder_block_unet(s1, 64)  
  s3 = _encoder_block_unet(s2, 128)  
  s4 = _encoder_block_unet(s3, 256)  
  # s4 = _encoder_block_unet(s3, 512)  
  # out = _encoder_block_unet(s4, 1024)  

  # out = _decoder_block_unet(out, 512)
  # out = _decoder_block_unet(out, 256)
  out = _decoder_block_unet(s4, 128)
  out = _decoder_block_unet(out, 64)
  out = _decoder_block_unet(out, 32)
  out = _decoder_block_unet(out, 16)
  out = _decoder_block_unet(out, num_classes)
  segnet_model = tf.keras.Model(inputs, out, name="SegNet")

  return segnet_model

# Import and process data

In [ ]:
x_train = import_data_numpy("/content/drive/MyDrive/data/resized/train/img")
y_train = import_data_numpy_mask("/content/drive/MyDrive/data/resized/train/labelcol")
x_test = import_data_numpy("/content/drive/MyDrive/data/resized/test/img")
y_test = import_data_numpy_mask("/content/drive/MyDrive/data/resized/test/labelcol")

In [ ]:
y_train_binary = np.copy(y_train)
y_test_binary = np.copy(y_test)

for i in range(len(y_train)):
  y_train_binary[i][y_train_binary[i] > 0] = 1  
  y_test_binary[i][y_test_binary[i] > 0] = 1 

In [ ]:
y_onehot_train = keras.utils.to_categorical(y_train_binary, 2)
y_onehot_test = keras.utils.to_categorical(y_test_binary, 2)

# Train models

In [ ]:
segnet_model = segnet((128,128,3), 2)

segnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(), 
    metrics=[f1_metric])
# Instaniate logging for tensorboard
segnet_train_log_dir = "logs/unet/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
segnet_train_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=segnet_train_log_dir, histogram_freq=1)

segnet_model.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    
                 callbacks=[segnet_train_tensorboard_callback]) 
segnet_model.save('segnet_model.h5', save_format='h5')

In [ ]:
unet_model = unet((128,128,3), 2)

unet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    
    loss=tf.keras.losses.BinaryCrossentropy(),  
    metrics=[f1_metric])

# Instaniate logging for tensorboard
unet_train_log_dir = "logs/unet/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
unet_train_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=unet_train_log_dir, histogram_freq=1)

unet_model.fit(x_train, y_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_test), verbose=1,   
                 callbacks=[unet_train_tensorboard_callback])    

unet_model.save('unet_model.h5', save_format='h5')

In [ ]:
kiunet_model = kiunet((128,128,3), 2)

kiunet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),   
    loss=tf.keras.losses.BinaryCrossentropy(),  
    metrics=[f1_metric])

# Instantiate logging for tensorboard
kiunet_train_log_dir = "logs/kiunet/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
kiunet_train_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=kiunet_train_log_dir, histogram_freq=1)


kiunet_model.fit(x_train, y_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_test), verbose=1,    
                 callbacks=[kiunet_train_tensorboard_callback])

kiunet_model.save('kiunet_model.h5', save_format='h5')

# 10 KiU-Net models experiment

In [ ]:
kiunet_model_1 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_1.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_1.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_1.save('kiUnet_model_1.h5', save_format='h5')

In [ ]:
kiunet_model_2 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_2.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_2.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_2.save('kiunet_model_2.h5', save_format='h5')

In [ ]:
kiunet_model_3 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_3.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_3.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_3.save('kiunet_model_3.h5', save_format='h5')

In [ ]:
kiunet_model_4 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_4.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_4.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_4.save('kiunet_model_4.h5', save_format='h5')

In [ ]:
kiunet_model_5 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_5.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_5.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_5.save('kiunet_model_5.h5', save_format='h5')

In [ ]:
kiunet_model_6 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_6.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_6.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_6.save('kiunet_model_6.h5', save_format='h5')

In [ ]:
kiunet_model_7 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_7.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_7.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_7.save('kiunet_model_7.h5', save_format='h5')

In [ ]:
kiunet_model_8 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_8.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_8.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_8.save('kiunet_model_8.h5', save_format='h5')

In [ ]:
kiunet_model_9 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_9.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_9.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_9.save('kiunet_model_9.h5', save_format='h5')

In [ ]:
kiunet_model_10 = kiunet((128,128,3), 2)
m = tf.keras.metrics.MeanIoU(num_classes=255)
# Save the model structure
# kiunet_model.save('kiUnet_structure')

kiunet_model_10.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),    ## learning_rate is different from 3D
    loss=tf.keras.losses.BinaryCrossentropy(),  ##maybe categorical_crossentropy if labels one-hot encoded
    metrics=[keras.metrics.MeanIoU(num_classes=255), f1_metric])#check metric, should be 4 classes

kiunet_model_10.fit(x_train, y_onehot_train, batch_size=1, epochs=300, 
                 validation_data=(x_test, y_onehot_test), verbose=1,    ## validation_data = test_data, no early stopping
                 callbacks=[kiunet_train_tensorboard_callback])    ## log metrics in TensorBoard


## Save again using H5 format
kiunet_model_10.save('kiunet_model_10.h5', save_format='h5')



# Evaluate models

In [ ]:
segnet_model = keras.models.load_model("segnet_model.h5",  custom_objects={'f1_metric': cm.f1_metric})
test_pred_segnet = segnet_model.predict(x_test)
y_classes_segnet = test_pred_segnet.argmax(axis=-1).flatten()
m = tf.keras.metrics.MeanIoU(num_classes=2)
y_onehot = y_onehot_test[:, :, :, 1].flatten()
m.update_state(y_classes_segnet, y_onehot)
segnet_dice = m.result().numpy()
print("\nSeg-Net Evaluation:")
print("Dice Score: {}".format(segnet_dice))
segnet_jaccard = f1_score(y_classes_segnet, y_onehot, average="macro")
print("Jaccard Score: {}".format(segnet_jaccard))

In [ ]:
unet_model = keras.models.load_model("unet_model.h5", custom_objects={'f1_metric': cm.f1_metric})
test_pred_unet = unet_model.predict(x_test)
y_classes_unet = test_pred_unet.argmax(axis=-1).flatten()
m = tf.keras.metrics.MeanIoU(num_classes=2)
y_onehot = y_onehot_test[:, :, :, 1].flatten()
m.update_state(y_classes_unet, y_onehot)
unet_dice = m.result().numpy()
print("\nU-Net Evaluation:")
print("Dice Score: {}".format(unet_dice))
unet_jaccard = f1_score(y_classes_unet, y_onehot, average="macro")
print("Jaccard Score: {}".format(unet_jaccard))

In [ ]:
kiunet_model = keras.models.load_model("kiUnet_model.h5", custom_objects={'f1_metric': cm.f1_metric})
test_pred_kiunet = kiunet_model.predict(x_test)
y_classes_kiunet = test_pred_kiunet.argmax(axis=-1).flatten()
m = tf.keras.metrics.MeanIoU(num_classes=2)
y_onehot = y_onehot_test[:, :, :, 1].flatten()
m.update_state(y_classes_kiunet, y_onehot)
kiunet_dice = m.result().numpy()
print("\nKiU-Net Evaluation:")
print("Dice Score: {}".format(kiunet_dice))
kiunet_jaccard = f1_score(y_classes_kiunet, y_onehot, average="macro")
print("Jaccard Score: {}".format(kiunet_jaccard))